In [1]:
# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000


In [3]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

mlflow.set_tracking_uri('http://localhost:5001') 

# set the experiment id
experiment_name = "busquedaHiperParamWAPredict"
mlflow.set_experiment(experiment_name)
"""
Codigo para mirar con Leo y Brian
#creo el experimento
experiment_name = ""busquedaHiperParamWAPredict""
if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name)
experiment = mlflow.get_experiment_by_name(experiment_name)
"""



mlflow.autolog()
db = load_diabetes()

X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
rf.fit(X_train, y_train)

# Use the model to make predictions on the test dataset.
predictions = rf.predict(X_test)



2025/04/05 13:11:25 INFO mlflow.bedrock: Enabled auto-tracing for Bedrock. Note that MLflow can only trace boto3 service clients that are created after this call. If you have already created one, please recreate the client by calling `boto3.client`.
2025/04/05 13:11:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for boto3.
2025/04/05 13:11:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/04/05 13:11:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f15036a9f0864372aeb1e9b1011c00f5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run wise-moose-795 at: http://localhost:5001/#/experiments/1/runs/f15036a9f0864372aeb1e9b1011c00f5
🧪 View experiment at: http://localhost:5001/#/experiments/1


Busqueda de hiperparametros

In [ ]:
"""
Se deja el codigo debajo a modo de plantilla, se debe actualizar con informacion especifica del modelo utilizado en el Tf Aprendiza Maquina  1
"""
model_wapredict = RandomForestClassifier() # cambiar a xgboost

# Definimos los hiperparámetros para la búsqueda
grid = {
    'max_depth':[6,8,10],
    'min_samples_split':[2,3,4,5],
    'min_samples_leaf':[2,3,4,5],
    'max_features': [2,3]
    }

# Hacemos la búsqueda
wapredict_grid = GridSearchCV(model, grid, cv=5)
wapredict_results = wapredict.fit(X_train, y_train)

print(f'Los mejores parámetros son: {wapredict.best_params_}')

In [ ]:
with mlflow.start_run(experiment_id = experiment.experiment_id):
    # Se registran los mejores hiperparámetros
    mlflow.log_params(wapredict_results.best_params_)

    # Se obtiene las predicciones del dataset de evaluación
    y_pred = wapredict_results.predict(X_test)

    # Se calculan las métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

    # Y las enviamos a MLFlow
    metrics ={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall
        }
    mlflow.log_metrics(metrics)